# Imports

In [1]:
import pandas as pd
import numpy as np
import re
pd.options.mode.chained_assignment = None

In [2]:
kl_data = pd.read_csv('kl_property.csv')

# Overview of the dataset

In [3]:
kl_data.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,"KLCC, Kuala Lumpur","RM 1,250,000",2+1,3.0,2.0,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,"Damansara Heights, Kuala Lumpur","RM 6,800,000",6,7.0,NaN,Bungalow,Land area : 6900 sq. ft.,Partly Furnished
2,"Dutamas, Kuala Lumpur","RM 1,030,000",3,4.0,2.0,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
3,"Cheras, Kuala Lumpur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Bukit Jalil, Kuala Lumpur","RM 900,000",4+1,3.0,2.0,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished


#### This dataset has 53883 records and 8 columns

In [4]:
kl_data.shape

(53883, 8)

#### Data type of each column

In [5]:
kl_data.dtypes

Location          object
Price             object
Rooms             object
Bathrooms        float64
Car Parks        float64
Property Type     object
Size              object
Furnishing        object
dtype: object

#### Number of null values in each column

In [6]:
kl_data.isna().sum()

Location             0
Price              248
Rooms             1706
Bathrooms         2013
Car Parks        17567
Property Type       25
Size              1063
Furnishing        6930
dtype: int64

# Data Preprocessing

### Price

Removing all records with missing Price value as Price will be used later on as the *target* for future prediction.

In [7]:
df = kl_data[kl_data['Price'].notnull()]

Price column has no missing values now.

In [8]:
df.isna().sum()

Location             0
Price                0
Rooms             1655
Bathrooms         1962
Car Parks        17441
Property Type        0
Size              1024
Furnishing        6856
dtype: int64

Converting Price values to be numeric.

In [9]:
def format_price(string):
    string = string.strip()
    string = string.replace("RM", "")
    string = string.replace(",", "")
    return int(string)

df['Price'] = df['Price'].apply(format_price)

In [10]:
df['Price'].dtype

dtype('int64')

### Location

Since this dataset contains only records of properties from Kuala Lumpur, we can remove the city name from all Location values.

In [11]:
df['Location'] = df['Location'].str.lower()
df['Location'] = df['Location'].str.replace(r", kuala lumpur$", "")

Large amount of unique location values making it hard to work with.

In [12]:
sorted(df['Location'].unique())

['adiva desa parkcity',
 'alam damai',
 'ampang',
 'ampang hilir',
 'bandar damai perdana',
 'bandar menjalara',
 'bandar sri damansara',
 'bandar tasik selatan',
 'bangsar',
 'bangsar south',
 'batu caves',
 'brickfields',
 'bukit  persekutuan',
 'bukit bintang',
 'bukit damansara',
 'bukit jalil',
 'bukit kiara',
 'bukit ledang',
 'bukit tunku (kenny hills)',
 'canary residence',
 'casa rimba',
 'chan sow lin',
 'cheras',
 'city centre',
 'country heights damansara',
 'cyberjaya',
 'damansara',
 'damansara heights',
 'desa pandan',
 'desa parkcity',
 'desa petaling',
 'duta nusantara',
 'dutamas',
 'federal hill',
 'gombak',
 'gurney',
 'happy garden',
 'jalan ipoh',
 'jalan klang lama (old klang road)',
 'jalan kuching',
 'jalan sultan ismail',
 'jalan u-thant',
 'jinjang',
 'kemensah',
 'kepong',
 'keramat',
 'kl city',
 'kl eco city',
 'kl sentral',
 'klcc',
 'kota damansara',
 'kuala lumpur',
 'kuchai lama',
 'landed sd',
 'mid valley city',
 'mont kiara',
 'off gasing indah,',
 

There are many locations with very few properties listed.

In [13]:
df['Location'].value_counts()[df['Location'].value_counts() < 100]

bandar tasik selatan    93
puchong                 82
damansara               75
pandan indah            63
jinjang                 54
mid valley city         50
sungai penchala         45
taman duta              41
federal hill            34
pandan jaya             25
other                   23
bukit ledang            14
chan sow lin            11
bukit kiara              9
gombak                   9
bandar sri damansara     7
sri damansara            3
santuari park pantai     3
kuala lumpur             3
taman yarl               2
adiva desa parkcity      2
happy garden             2
taman sri keramat        2
canary residence         1
taman connaught          1
semarak                  1
kota damansara           1
taman melati             1
singapore                1
the mines resort         1
u-thant                  1
rawang                   1
gurney                   1
off gasing indah,        1
solaris dutamas          1
taman ibukota            1
sri kembangan            1
c

Here, we consider a location to be **significant** if it has more than 100 properties listed.

In [14]:
significant_loc = df['Location'].value_counts()[df['Location'].value_counts() >= 100].index
significant_loc

Index(['mont kiara', 'klcc', 'cheras', 'jalan klang lama (old klang road)',
       'setapak', 'bukit jalil', 'kepong', 'bangsar', 'desa parkcity',
       'damansara heights', 'dutamas', 'ampang', 'sentul', 'sungai besi',
       'taman tun dr ismail', 'sri hartamas', 'taman desa', 'wangsa maju',
       'segambut', 'bukit bintang', 'kl city', 'kuchai lama', 'sri petaling',
       'jalan kuching', 'kl sentral', 'jalan ipoh', 'taman melawati',
       'ampang hilir', 'city centre', 'bukit tunku (kenny hills)',
       'bandar menjalara', 'bangsar south', 'seputeh', 'pantai', 'oug',
       'batu caves', 'country heights damansara', 'brickfields', 'sunway spk',
       'keramat', 'salak selatan', 'titiwangsa', 'kl eco city',
       'desa petaling', 'bandar damai perdana', 'setiawangsa', 'desa pandan',
       'jalan sultan ismail', 'pandan perdana'],
      dtype='object')

Filtering the records to contain **only** significant locations.

In [15]:
df = df.loc[np.isin(df['Location'], significant_loc)]

This is how the data frame looks like after cleaning Location and Price columns.

In [16]:
df.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,klcc,1250000,2+1,3.0,2.0,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,damansara heights,6800000,6,7.0,NaN,Bungalow,Land area : 6900 sq. ft.,Partly Furnished
2,dutamas,1030000,3,4.0,2.0,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
4,bukit jalil,900000,4+1,3.0,2.0,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished
5,taman tun dr ismail,5350000,4+2,5.0,4.0,Bungalow,Land area : 7200 sq. ft.,Partly Furnished


### Rooms

The values in Rooms are either 'x', 'x+', 'x+y', 'x Above' or 'Studio', where x and y are integers in the form of a string. We will format the strings and convert it into integer values.

In [17]:
sorted(df['Rooms'].unique().astype(str))

['1',
 '1+1',
 '1+2',
 '10',
 '10+',
 '10+1',
 '11',
 '11+1',
 '12',
 '12+',
 '13',
 '13+',
 '13+1',
 '14',
 '15+',
 '16',
 '18',
 '2',
 '2+1',
 '2+2',
 '20 Above',
 '3',
 '3+1',
 '3+2',
 '4',
 '4+1',
 '4+2',
 '5',
 '5+1',
 '5+2',
 '6',
 '6+',
 '6+1',
 '7',
 '7+',
 '7+1',
 '8',
 '8+',
 '8+1',
 '9',
 '9+',
 '9+1',
 'Studio',
 'nan']

We will be using regular expressions in order to match the strings. Studio is assumed to be 1 room.

In [18]:
def format_rooms(string):
    try:
        if re.search("[0-9]+ above", string.lower()) is not None:
            return int(string.split(' ')[0])

        elif re.search("[0-9]+\+[0-9]+", string) is not None:
            num_list = string.split("+")
            return int(num_list[0]) + int(num_list[1])

        elif re.search("\+$", string) is not None:
            return int(string[:-1])

        elif string.lower() == 'studio':
            return 1

        else:
            return int(string)
    
    except AttributeError:
        return string
    
    
df['Rooms'] = df['Rooms'].apply(format_rooms)

In [19]:
df['Rooms'].dtype

dtype('float64')

### Property Type

The full list of property types are displayed and it is overwhelming at first. We are going to categorise these property types into their respective super-group.

In [20]:
sorted(df['Property Type'].unique())

['1-sty Terrace/Link House',
 '1-sty Terrace/Link House (Corner)',
 '1-sty Terrace/Link House (EndLot)',
 '1-sty Terrace/Link House (Intermediate)',
 '1.5-sty Terrace/Link House',
 '1.5-sty Terrace/Link House (Corner)',
 '1.5-sty Terrace/Link House (EndLot)',
 '1.5-sty Terrace/Link House (Intermediate)',
 '2-sty Terrace/Link House',
 '2-sty Terrace/Link House (Corner)',
 '2-sty Terrace/Link House (Duplex)',
 '2-sty Terrace/Link House (EndLot)',
 '2-sty Terrace/Link House (Intermediate)',
 '2-sty Terrace/Link House (Penthouse)',
 '2.5-sty Terrace/Link House',
 '2.5-sty Terrace/Link House (Corner)',
 '2.5-sty Terrace/Link House (Duplex)',
 '2.5-sty Terrace/Link House (EndLot)',
 '2.5-sty Terrace/Link House (Intermediate)',
 '2.5-sty Terrace/Link House (Penthouse)',
 '2.5-sty Terrace/Link House (Triplex)',
 '3-sty Terrace/Link House',
 '3-sty Terrace/Link House (Corner)',
 '3-sty Terrace/Link House (Duplex)',
 '3-sty Terrace/Link House (EndLot)',
 '3-sty Terrace/Link House (Intermediate)'

These are the super-groups identified.

In [21]:
main_types = ["Townhouse",
              "Semi-detached House",
              "Serviced Residence",
              "Residential Land",
              "Flat",
              "Condominium",
              "Cluster House",
              "Bungalow",
              "Apartment",
              "Terrace/Link House"]

In [22]:
def format_prop_type(string):
    for main in main_types:
        if re.search(main, string, flags=re.IGNORECASE) is not None:
            return main
    return string

A new column **Property Type Supergroup** is added to the data frame. Here's how the data frame looks like so far.

In [23]:
df['Property Type Supergroup'] = df['Property Type'].apply(format_prop_type)

In [24]:
df.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,Property Type Supergroup
0,klcc,1250000,3.0,3.0,2.0,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished,Serviced Residence
1,damansara heights,6800000,6.0,7.0,NaN,Bungalow,Land area : 6900 sq. ft.,Partly Furnished,Bungalow
2,dutamas,1030000,3.0,4.0,2.0,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished,Condominium
4,bukit jalil,900000,5.0,3.0,2.0,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished,Condominium
5,taman tun dr ismail,5350000,6.0,5.0,4.0,Bungalow,Land area : 7200 sq. ft.,Partly Furnished,Bungalow


There are significantly more condominiums and serviced residences in Kuala Lumpur.

In [25]:
df['Property Type Supergroup'].value_counts()

Condominium            23149
Serviced Residence     13399
Terrace/Link House      7567
Bungalow                3598
Semi-detached House     2102
Apartment               1621
Residential Land         669
Townhouse                508
Flat                     330
Cluster House              6
Name: Property Type Supergroup, dtype: int64

### Size

The values of Size are in this format: [Built-up/Land area] : [size] sq. ft.

We are only interested in the [size] information and we need to convert Size column to be numeric.

In [26]:
def split_size(string, idx):
    try:
        return string.split(":")[idx].strip().strip("sq. ft.")
    except AttributeError:
        return string

    
df['Size'] = df['Size'].apply(split_size, idx = 1)

In [27]:
def format_size(string):
        
    try:
        
        string = string.lower()
        string = string.replace(" ", "")
        string = string.replace("'", "")
        string = string.replace(",", "")
        
        # match [## + ## x ##]
        match = re.search('(\d+)\+(\d+)x(\d+)', string)
        if match:
            return int(match.group(1)) + (int(match.group(2)) * int(match.group(3)))
        
        # match [## x ##] or [## * ##]
        match2 = re.search('(\d+)(x|\*)(\d+)', string)
        if match2:
            return int(match2.group(1)) * int(match2.group(3))

        return int(string)
    
    except:
        
        # this guarantee that Size column is numeric
        return None
    

df['Size'] = df['Size'].apply(format_size)

It is important that we remove records with missing Size values as we will be needing this later for feature engineering.

In [28]:
df = df[~df['Size'].isna()]

In [29]:
df['Size'].dtype

dtype('float64')

Here is how the data frame looks like so far.

In [30]:
df.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,Property Type Supergroup
0,klcc,1250000,3.0,3.0,2.0,Serviced Residence,1335.0,Fully Furnished,Serviced Residence
1,damansara heights,6800000,6.0,7.0,NaN,Bungalow,6900.0,Partly Furnished,Bungalow
2,dutamas,1030000,3.0,4.0,2.0,Condominium (Corner),1875.0,Partly Furnished,Condominium
4,bukit jalil,900000,5.0,3.0,2.0,Condominium (Corner),1513.0,Partly Furnished,Condominium
5,taman tun dr ismail,5350000,6.0,5.0,4.0,Bungalow,7200.0,Partly Furnished,Bungalow


### Bathrooms and Car Parks

Bathrooms and Car Parks columns requires no further cleaning.

In [31]:
df['Bathrooms'].value_counts()

2.0     20548
3.0      9157
4.0      6417
5.0      4647
1.0      4366
6.0      3137
7.0      1018
8.0       395
9.0       139
10.0       70
12.0       14
11.0       11
20.0       10
15.0        6
13.0        5
14.0        4
16.0        1
Name: Bathrooms, dtype: int64

In [32]:
df['Car Parks'].value_counts()

2.0     14787
1.0     13286
3.0      3720
4.0      1881
6.0       617
5.0       497
8.0        88
10.0       76
7.0        62
12.0        7
15.0        7
13.0        7
28.0        4
20.0        4
9.0         4
30.0        4
11.0        4
16.0        2
17.0        2
18.0        1
24.0        1
Name: Car Parks, dtype: int64

# Feature Engineering

Price per Size is commonly used to standardise pricing of properties. We will be using this as the target data for future predictions later on.

In [33]:
df['Price per Size'] = df['Price'] / df['Size']

---

The final data frame after pre-processing and feature engineering.

In [34]:
df.head(10)

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,Property Type Supergroup,Price per Size
0,klcc,1250000,3.0,3.0,2.0,Serviced Residence,1335.0,Fully Furnished,Serviced Residence,936.329588
1,damansara heights,6800000,6.0,7.0,NaN,Bungalow,6900.0,Partly Furnished,Bungalow,985.507246
2,dutamas,1030000,3.0,4.0,2.0,Condominium (Corner),1875.0,Partly Furnished,Condominium,549.333333
4,bukit jalil,900000,5.0,3.0,2.0,Condominium (Corner),1513.0,Partly Furnished,Condominium,594.844679
5,taman tun dr ismail,5350000,6.0,5.0,4.0,Bungalow,7200.0,Partly Furnished,Bungalow,743.055556
7,taman tun dr ismail,2600000,5.0,4.0,4.0,Semi-detached House,3600.0,Partly Furnished,Semi-detached House,722.222222
8,taman tun dr ismail,1950000,5.0,4.0,3.0,2-sty Terrace/Link House (EndLot),1875.0,Partly Furnished,Terrace/Link House,1040.000000
9,sri petaling,385000,3.0,2.0,1.0,Apartment (Intermediate),904.0,Partly Furnished,Apartment,425.884956
11,taman tun dr ismail,1680000,4.0,3.0,NaN,2-sty Terrace/Link House (Intermediate),1760.0,Partly Furnished,Terrace/Link House,954.545455
12,taman tun dr ismail,1700000,4.0,3.0,NaN,2-sty Terrace/Link House (Intermediate),1900.0,Partly Furnished,Terrace/Link House,894.736842


# Save cleaned data

In [35]:
df.to_csv('kl_property_cleaned.csv')